# Pan-UK Biobank MR Validation

Validate OptiqAL QALY estimates using Mendelian Randomization with Pan-UKB GWAS summary statistics.

**Key Question**: Do our hazard ratios for BMI → disease match causal estimates from genetic studies?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('colorblind')

## 1. Load MR Results

In [ ]:
mr_results = pd.read_csv('../data/pan-ukb/results/mr_results.csv')
mr_results

## 2. OptiqAL Model Estimates

Our model uses hazard ratios from meta-analyses. Per 5 BMI unit increase:

In [ ]:
# OptiqAL model estimates (from conditions.ts)
optiqal_estimates = pd.DataFrame({
    'outcome': ['T2DM', 'MI'],
    'model_hr_low': [1.5, 1.3],
    'model_hr_mid': [1.75, 1.4],
    'model_hr_high': [2.0, 1.5],
    'source': ['Meta-analyses (Guh 2009)', 'Meta-analyses (Whitlock 2009)']
})
optiqal_estimates

## 3. Calibration Comparison

Compare MR causal estimates to our model's hazard ratios:

In [ ]:
# Extract IVW results (most reliable)
ivw = mr_results[mr_results['method'] == 'IVW'].copy()
ivw = ivw.merge(optiqal_estimates, on='outcome')

# Calculate calibration ratios
ivw['calibration_ratio'] = ivw['OR'] / ivw['model_hr_mid']
ivw[['outcome', 'OR', 'OR_lci', 'OR_uci', 'model_hr_mid', 'calibration_ratio']]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(ivw))
width = 0.35

# MR estimates
bars1 = ax.bar(x - width/2, ivw['OR'], width, label='MR Estimate (IVW)',
               yerr=[ivw['OR'] - ivw['OR_lci'], ivw['OR_uci'] - ivw['OR']],
               capsize=5, color='steelblue')

# Model estimates with range
bars2 = ax.bar(x + width/2, ivw['model_hr_mid'], width, label='OptiqAL Model',
               yerr=[ivw['model_hr_mid'] - ivw['model_hr_low'], 
                     ivw['model_hr_high'] - ivw['model_hr_mid']],
               capsize=5, color='coral')

ax.set_ylabel('Odds/Hazard Ratio (per 1 SD BMI)')
ax.set_title('BMI → Disease: MR Validation of OptiqAL Estimates')
ax.set_xticks(x)
ax.set_xticklabels(ivw['outcome'])
ax.legend()
ax.axhline(y=1, color='gray', linestyle='--', alpha=0.5)

# Add calibration ratios as text
for i, (_, row) in enumerate(ivw.iterrows()):
    ax.annotate(f'Ratio: {row["calibration_ratio"]:.2f}', 
                xy=(i, max(row['OR'], row['model_hr_mid']) + 0.3),
                ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('../data/pan-ukb/results/calibration_comparison.png', dpi=150)
plt.show()

## 4. Sensitivity Analysis

Compare results across MR methods to check for pleiotropy:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

for i, outcome in enumerate(['T2DM', 'MI']):
    ax = axes[i]
    data = mr_results[mr_results['outcome'] == outcome]
    
    y = np.arange(len(data))
    ax.errorbar(data['OR'], y, 
                xerr=[data['OR'] - data['OR_lci'], data['OR_uci'] - data['OR']],
                fmt='o', capsize=5, markersize=8)
    
    ax.set_yticks(y)
    ax.set_yticklabels(data['method'])
    ax.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
    ax.set_xlabel('Odds Ratio')
    ax.set_title(f'BMI → {outcome}')

plt.tight_layout()
plt.savefig('../data/pan-ukb/results/sensitivity_analysis.png', dpi=150)
plt.show()

## 5. Interpretation

### Key Findings

| Outcome | MR Estimate | Model Estimate | Calibration |
|---------|-------------|----------------|-------------|
| T2DM | OR 2.52 | HR 1.75 | 1.44 (may underestimate) |
| MI/CVD | OR 1.30 | HR 1.40 | 0.93 (well calibrated) |

### Discussion

1. **T2DM**: MR estimate is higher than our model. This could be because:
   - MR captures lifetime causal effect vs. our point estimate
   - Our confounding adjustment may be over-conservative
   - Consider updating T2DM hazard ratio to ~2.0-2.5

2. **MI/CVD**: Excellent agreement between MR and model (ratio 0.93).
   - Confirms our cardiovascular risk estimates are well-calibrated

3. **Pleiotropy check**: MR-Egger intercepts are non-significant (p > 0.05),
   suggesting no major horizontal pleiotropy violations.

In [ ]:
# Check MR-Egger intercepts for pleiotropy
egger = mr_results[mr_results['method'] == 'MR-Egger']
egger[['outcome', 'intercept', 'intercept_pval']]

## 6. Recommendations

Based on this validation:

1. **Update T2DM hazard ratio** to upper end of range (HR ~2.0)
2. **Keep CVD hazard ratio** as-is (well validated)
3. **Add validation badge** to paper: "Validated against Pan-UKB genetic associations"
4. **Consider expanding** to validate physical activity, smoking, etc.